<a href="https://colab.research.google.com/github/OdysseusPolymetis/chr2023/blob/main/5_example_ugarit_multilingualSimilarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center>**Measuring Greek and Latin similarity**</center>

Requirements

In [1]:
!pip install torch transformers numpy stanza umap-learn sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 933.2/933.2 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.5/397.5 kB 39.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 8.6 MB/s eta 0:00:00
  Created wheel for umap-learn: filename=umap_learn-0.5.5-py3-none-any.whl size=86832 sha256=bb47ac46b25271d355eec5d758dcf8d37b4d0f6e5bc1582ebebae9de1339d3b4
  Stored in directory: /root/.cache/pip/wheels/3a/70/07/428d2b58660a1a3b431db59b806a10da736612ebbc66c1bcc5
Successfully built umap-learn


In [2]:
!git clone https://github.com/PerseusDL/canonical-greekLit.git

Cloning into 'canonical-greekLit'...
remote: Enumerating objects: 60517, done.
remote: Counting objects: 100% (95/95), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 60517 (delta 50), reused 76 (delta 39), pack-reused 60422
Receiving objects: 100% (60517/60517), 532.14 MiB | 15.77 MiB/s, done.
Resolving deltas: 100% (40203/40203), done.
Updating files: 100% (2631/2631), done.


In [3]:
!git clone https://github.com/PerseusDL/canonical-latinLit.git

Cloning into 'canonical-latinLit'...
remote: Enumerating objects: 25683, done.
remote: Counting objects: 100% (921/921), done.
remote: Compressing objects: 100% (421/421), done.
remote: Total 25683 (delta 515), reused 892 (delta 496), pack-reused 24762
Receiving objects: 100% (25683/25683), 301.21 MiB | 15.88 MiB/s, done.
Resolving deltas: 100% (16178/16178), done.


Imports

In [2]:
import glob
from joblib import Parallel, delayed
import pickle
from collections import defaultdict
import os
import numpy as np
import re
from lxml import etree
import stanza
from tqdm import tqdm
import unicodedata

In [3]:
import torch
from transformers import pipeline
from transformers import BertTokenizer, BertModel
pipe = pipeline("fill-mask", model="UGARIT/grc-alignment")
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("UGARIT/grc-alignment")
model = AutoModelForMaskedLM.from_pretrained("UGARIT/grc-alignment")
model.eval()  # Mode évaluation

Some weights of the model checkpoint at UGARIT/grc-alignment were not used when initializing XLMRobertaForMaskedLM: ['psi_cls.bias', 'psi_cls.transform.weight', 'psi_cls.decoder.weight', 'psi_cls.transform.bias']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at UGARIT/grc-alignment were not used when initializing XLMRobertaForMaskedLM: ['psi_cls.bias', 'psi_cls.transform.weight', 'psi_cls.decoder.weight', 'psi_cls.transform.bias']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model t

XLMRobertaForMaskedLM(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
  

In [4]:
print(tokenizer.tokenize("un exemple avec accent ὡἤ"))

['▁un', '▁exemple', '▁avec', '▁accent', '▁', 'ὡ', 'ἤ']


parcours des xml

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

XLMRobertaForMaskedLM(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
  

In [6]:
def extract_body_content_from_xml(file_path):
    parser = etree.XMLParser(recover=True)
    tree = etree.parse(file_path, parser)
    nsmap = tree.getroot().nsmap
    default_ns = nsmap.get(None)

    if default_ns:
        body = tree.find(".//ns:body", namespaces={"ns": default_ns})
    else:
        body = tree.find(".//body")

    if body is None:
        raise ValueError(f"No <body> element found in {file_path}")

    return etree.tostring(body, method="text", encoding="unicode")

def is_latin(filename):
    return re.search(r'lat\d+\.xml$', filename) is not None

def is_greek(filename):
    return re.search(r'grc\d+\.xml$', filename) is not None

def extract_target_texts_from_directory(directory_path, target_authors):
    texts = {}
    for root, _, files in os.walk(directory_path):
        for file in files:
            if file.endswith('.xml') and any(author_tag in file for author_tag in target_authors):
                file_path = os.path.join(root, file)
                try:
                    content = extract_body_content_from_xml(file_path)
                    texts[file] = content
                except Exception as e:
                    print(f"Erreur lors du traitement du fichier {file_path}: {e}")
    return texts


In [7]:
grc_target_authors = ["tlg0059","tlg0086","tlg1325", "tlg0626","tlg1304","tlg0632","tlg0591","tlg0593","tlg1562","tlg1705","tlg0014","tlg0610"]
lat_target_authors = ["phi0474", "phi1017","stoa0255","phi1014","tlg0557","phi0550","tlg0628","tlg0562","phi1254","phi1002","stoa0058","phi0684","phi1212"]


In [8]:
greek_corpus = extract_target_texts_from_directory('/content/canonical-greekLit', grc_target_authors)
latin_corpus = extract_target_texts_from_directory('/content/canonical-latinLit', lat_target_authors)

In [9]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

def segment_corpus_into_sentences(corpus):
    segmented_corpus = {}
    for text_id, text in corpus.items():
        segmented_corpus[text_id] = segment_into_sentences_with_index(text)
    return segmented_corpus


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
def segment_into_sentences_with_index(text):
    sentences = sent_tokenize(text)
    return {i: sentence for i, sentence in enumerate(sentences)}

In [11]:
greek_corpus_segmented = segment_corpus_into_sentences(greek_corpus)
latin_corpus_segmented = segment_corpus_into_sentences(latin_corpus)

In [12]:
def encode_sentences_with_ugarit(sentences_dict, tokenizer, model, device, batch_size=32):
    sentences_batches = [list(sentences_dict.values())[i:i + batch_size] for i in range(0, len(sentences_dict), batch_size)]

    encoded_sentences = {}
    sentence_index = 0

    for batch in sentences_batches:
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=128)
        inputs = {key: value.to(device) for key, value in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs, output_hidden_states=True)

        for i in range(outputs.hidden_states[-1].shape[0]):
            sentence_embeddings = outputs.hidden_states[-1][i].mean(dim=0).squeeze(0)
            encoded_sentences[sentence_index] = sentence_embeddings
            sentence_index += 1

    return encoded_sentences


In [13]:
def encode_corpus_sentences(corpus_segmented, tokenizer, model, device, batch_size):
    corpus_embeddings = {}
    for text_id, sentences in tqdm(corpus_segmented.items(), desc="Encodage du corpus"):
        corpus_embeddings[text_id] = encode_sentences_with_ugarit(sentences, tokenizer, model, device, batch_size)
    return corpus_embeddings

In [ ]:
batch_size = 64
encoded_greek_corpus = encode_corpus_sentences(greek_corpus_segmented, tokenizer, model, device, batch_size)
encoded_latin_corpus = encode_corpus_sentences(latin_corpus_segmented, tokenizer, model, device, batch_size)

Encodage du corpus:  61%|██████    | 94/154 [05:51<08:40,  8.67s/it]

In [30]:
from sklearn.metrics.pairwise import cosine_similarity

def calculate_sentence_similarity(greek_sentence_embeddings, latin_sentence_embeddings):
    similarities = {}
    for greek_id, greek_emb in tqdm(greek_sentence_embeddings.items(), desc="Calcul des similarités"):
        greek_emb_cpu = greek_emb.to('cpu').numpy()  # Déplacer sur le CPU et convertir en numpy
        for latin_id, latin_emb in latin_sentence_embeddings.items():
            latin_emb_cpu = latin_emb.to('cpu').numpy()  # Déplacer sur le CPU et convertir en numpy
            sim = cosine_similarity([greek_emb_cpu], [latin_emb_cpu])[0][0]
            similarities[(greek_id, latin_id)] = sim
    return similarities


In [ ]:
def find_best_matches_for_each_text_pair(corpus_greek, corpus_latin, encoded_corpus_greek, encoded_corpus_latin):
    best_matches = {}

    for greek_text_id in corpus_greek:
        for latin_text_id in corpus_latin:
            greek_sentences = encoded_corpus_greek[greek_text_id]
            latin_sentences = encoded_corpus_latin[latin_text_id]

            # Calcul des similarités pour les paires de phrases
            similarity_scores = calculate_sentence_similarity(greek_sentences, latin_sentences)

            # Trouver la meilleure correspondance pour chaque paire de textes
            best_match = max(similarity_scores.items(), key=lambda x: x[1])
            best_matches[(greek_text_id, latin_text_id)] = best_match

    return best_matches


In [ ]:
best_matches = find_best_matches_for_each_text_pair(greek_corpus_segmented, latin_corpus_segmented, encoded_greek_corpus, encoded_latin_corpus)

for ((greek_text_id, latin_text_id), ((greek_sentence_id, latin_sentence_id), sim_score)) in best_matches.items():
    greek_sentence = greek_corpus_segmented[greek_text_id][greek_sentence_id]
    latin_sentence = latin_corpus_segmented[latin_text_id][latin_sentence_id]

    print(f"Meilleure correspondance entre le texte grec {greek_text_id} et le texte latin {latin_text_id}:")
    print(f"Phrase grecque: \"{greek_sentence}\"")
    print(f"Phrase latine: \"{latin_sentence}\"")
    print(f"Score de similarité: {sim_score}\n")
